<a href="https://colab.research.google.com/github/bominwang/Bayesian-statistics-method/blob/Bayesian-model-calibration/BC_ADVI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import scipy
import matplotlib
from matplotlib import pyplot as plt
from scipy import stats

In [3]:
!pip install pymc==4.1.4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 543.1/543.1 KB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 70.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 KB 6.4 MB/s eta 0:00:00
  Attempting uninstall: pymc
    Found existing installation: pymc 5.1.2
    Uninstalling pymc-5.1.2:
      Successfully uninstalled pymc-5.1.2


In [7]:
import pymc as pm
pm.__version__

'4.1.4'

In [8]:
# 构建物理模型，例如：
def model(x, a, b):
    return a * x + b

# 模拟观测数据，例如：
x_obs = np.array([1, 2, 3, 4, 5])
y_obs = np.array([2.1, 4.1, 6.2, 8.2, 10.3])

with pm.Model() as my_model:
    # 定义先验分布
    a = pm.Normal('a', mu=0, sigma=10)
    b = pm.Normal('b', mu=0, sigma=10)
    
    # 定义似然函数
    y = pm.Normal('y', mu=model(x_obs, a, b), sigma=0.1, observed=y_obs)
    
    # 进行变分推断
    approx = pm.fit(method='advi')
    
    # 输出结果
    trace = approx.sample(draws=5000)
    pm.summary(trace)

Shape validation failed: input_shape: (1, 5000), minimum_shape: (chains=2, draws=4)




```
import pymc3 as pm
import numpy as np

# 自定义似然函数
def custom_likelihood(y_obs, y_pred, sigma):
    # 这里使用了正态分布作为似然函数
    likelihood = pm.Normal("likelihood", mu=y_pred, sigma=sigma, observed=y_obs)
    return likelihood

# 定义物理模型
def model(theta):
    # 这里可以根据具体问题来定义模型
    y = theta[0] * np.sin(theta[1]) + theta[2] * np.cos(theta[3])
    return y

# 生成观测数据
np.random.seed(42)
n = 100
theta_true = np.array([1.0, np.pi/4, 0.5, np.pi/3])
epsilon = 0.1
X = np.random.rand(n, 4) * 10.0
y_obs = model(theta_true) + np.random.randn(n) * epsilon

# 定义先验分布
with pm.Model() as model:
    theta = pm.Normal('theta', mu=0, sigma=10, shape=4)
    sigma = pm.HalfNormal('sigma', sigma=1)

    # 定义似然函数
    y_pred = model(theta)
    likelihood = custom_likelihood(y_obs, y_pred, sigma)

    # 进行变分推断
    approx = pm.fit(method='advi')

    # 获取后验分布
    trace = approx.sample(1000)

# 输出后验分布的统计信息
pm.summary(trace)
```
在上述代码中，自定义的似然函数为custom_likelihood，它的参数包括观测数据y_obs、模型预测结果y_pred和标准差sigma。在定义模型时，我们将先验分布和似然函数都加入到了with pm.Model() as model:的上下文管理器中，使用pm.Normal和pm.HalfNormal定义了参数的先验分布，使用custom_likelihood定义了似然函数。最后使用pm.fit进行变分推断，获取模型参数的后验分布。

需要注意的是，在定义自定义的似然函数时，需要遵循pymc3库中定义似然函数的规范，将其定义为一个pymc3的随机变量，并使用observed参数指定观测数据。


在利用自动微分变分推断修正物理模型参数时，可以使用pm.Potential函数将参数限制在一定范围内。下面是一个示例程序：


```
import pymc3 as pm
import numpy as np

# 定义物理模型
def model(theta):
    # 这里可以根据具体问题来定义模型
    y = theta[0] * np.sin(theta[1]) + theta[2] * np.cos(theta[3])
    return y

# 生成观测数据
np.random.seed(42)
n = 100
theta_true = np.array([1.0, np.pi/4, 0.5, np.pi/3])
epsilon = 0.1
X = np.random.rand(n, 4) * 10.0
y_obs = model(theta_true) + np.random.randn(n) * epsilon

# 定义先验分布
with pm.Model() as model:
    theta = pm.Uniform('theta', lower=0, upper=1, shape=4)
    sigma = pm.HalfNormal('sigma', sigma=1)

    # 定义似然函数
    y_pred = model(theta)
    likelihood = pm.Normal('likelihood', mu=y_pred, sigma=sigma, observed=y_obs)

    # 进行变分推断
    approx = pm.fit(method='advi')

    # 获取后验分布
    trace = approx.sample(1000)

# 输出后验分布的统计信息
pm.summary(trace)
```

在上述代码中，我们使用pm.Uniform定义了参数的先验分布，并通过lower和upper参数指定了参数的取值范围。这里我们将参数的取值范围限制在0到1之间。在定义似然函数时，我们直接使用了pm.Normal来定义正态分布的似然函数。

需要注意的是，对参数取值范围的限制应该根据具体问题进行调整，以确保先验分布和观测数据的一致性。另外，当参数的取值范围非常大时，使用先验分布来限制参数取值的效果可能不如使用约束条件等其他方法。




```
import pymc3 as pm
import numpy as np

# 定义物理模型
def model(theta):
    # 这里可以根据具体问题来定义模型
    y = theta[0] * np.sin(theta[1]) + theta[2] * np.cos(theta[3])
    return y

# 生成观测数据
np.random.seed(42)
n = 100
theta_true = np.array([1.0, np.pi/4, 0.5, np.pi/3])
epsilon = 0.1
X = np.random.rand(n, 4) * 10.0
y_obs = model(theta_true) + np.random.randn(n) * epsilon

# 定义先验分布
with pm.Model() as model:
    theta_0 = pm.Normal('theta_0', mu=0, sigma=1)
    theta_1 = pm.Normal('theta_1', mu=0, sigma=1)
    theta_2 = pm.Normal('theta_2', mu=0, sigma=1)
    theta_3 = pm.Normal('theta_3', mu=0, sigma=1)

    theta = pm.Bound(pm.Normal, lower=0, upper=1, shape=4)('theta', mu=[theta_0, theta_1, theta_2, theta_3])

    sigma = pm.HalfNormal('sigma', sigma=1)

    # 定义似然函数
    y_pred = model(theta)
    likelihood = pm.Normal('likelihood', mu=y_pred, sigma=sigma, observed=y_obs)

    # 进行变分推断
    approx = pm.fit(method='advi')

    # 获取后验分布
    trace = approx.sample(1000)

# 输出后验分布的统计信息
pm.summary(trace)
```

在上述代码中，我们使用pm.Normal定义了四个参数的先验分布，并通过pm.Bound将参数的取值范围限制在0到1之间。需要注意的是，pm.Bound需要传入两个参数，第一个参数是要限制的分布类型，第二个参数是分布的参数，因此我们需要使用shape=4来指定参数的个数，同时将四个参数分别作为pm.Normal的参数传入。





```
import pymc3 as pm
import numpy as np
import theano.tensor as tt

# 定义自定义的似然函数
def my_likelihood(x, y_obs, theta):
    y_pred = theta[0] * x ** 2 + theta[1] * x + theta[2] * tt.sin(theta[3] * x) + theta[4]
    epsilon = theta[5]
    return pm.Normal.dist(mu=y_pred, sigma=epsilon).logp(y_obs).sum()

# 生成观测数据
np.random.seed(42)
n = 50
x_obs = np.linspace(-5, 5, n)
theta_true = np.array([0.5, -1.0, 2.0, 0.5, 1.0, 0.5])
y_obs = theta_true[0] * x_obs ** 2 + theta_true[1] * x_obs + theta_true[2] * np.sin(theta_true[3] * x_obs) + theta_true[4] + np.random.randn(n) * theta_true[5]

# 定义模型
with pm.Model() as model:
    # 定义参数的先验分布
    theta = pm.Uniform('theta', lower=-10.0, upper=10.0, shape=6)
    # 定义似然函数
    likelihood = pm.Potential('likelihood', my_likelihood(x_obs, y_obs, theta))
    # 进行变分推断
    approx = pm.fit(method='advi')
    trace = approx.sample(1000)

# 输出参数的后验分布的统计信息
pm.summary(trace)
```



当需要处理更多的参数时，可以使用shape参数将参数数目扩展为所需的数量，并根据需要对每个参数添加先验分布。

如果需要自定义一个较为复杂的似然函数，可以使用pm.DensityDist来定义一个密度函数，然后将其作为似然函数使用。下面是一个示例程序，演示了如何使用pymc3中的pm.DensityDist定义一个自定义的似然函数，同时优化5个参数：



```
import pymc3 as pm
import numpy as np

# 定义物理模型
def model(x, theta):
    y = theta[0] * np.sin(theta[1] * x) + theta[2] * np.exp(-theta[3] * x) + theta[4]
    return y

# 定义自定义的似然函数
def custom_likelihood(y_obs, y_pred):
    diff = y_obs - y_pred
    loglikelihood = -0.5 * np.sum(diff ** 2)
    return loglikelihood

# 生成观测数据
np.random.seed(42)
n = 50
x_obs = np.linspace(0, 10, n)
theta_true = np.array([1.0, 0.5, 0.5, 0.1, 0.5])
epsilon = 0.5
y_obs = model(x_obs, theta_true) + np.random.randn(n) * epsilon

# 定义模型
with pm.Model() as model:
    # 定义参数的先验分布
    theta = pm.Uniform('theta', lower=0.0, upper=1.0, shape=5)
    # 定义似然函数
    y_pred = model(x_obs, theta)
    likelihood = pm.DensityDist('likelihood', logp=lambda y_pred: custom_likelihood(y_obs, y_pred), observed=y_pred)
    # 进行变分推断
    approx = pm.fit(method='advi')
    trace = approx.sample(1000)

# 输出参数的后验分布的统计信息
pm.summary(trace)
```


在上述代码中，我们定义了一个包含5个参数的物理模型，并定义了一个自定义的似然函数custom_likelihood，它将预测值和观测值作为输入，并返回对数似然。然后我们使用pm.DensityDist将其作为似然函数使用。

需要注意的是，在自定义似然函数时，必须使用对数似然而不是原始似然，因为pymc3只能处理对数似然。另外，自定义似然函数必须接受预测值和观测值作为输入，并返回对数似然。

在定义参数的先验分布时，我们使用pm.Uniform将每个参数的取值限制在0和1之间。需要根据具体问题来调整参数的先验分布，以确保先验分布和观测数据的一致性。



```
import pymc3 as pm
import numpy as np

# 定义物理模型
def model(x, theta):
    y = theta[0] * np.sin(theta[1] * x) + theta[2] * x ** 2 + theta[3] * x + theta[4]
    return y

# 生成观测数据
np.random.seed(42)
n = 50
x_obs = np.linspace(-5, 5, n)
theta_true = np.array([1.0, 2.0, -1.0, 0.5, 3.0])
epsilon = 0.5
y_obs = model(x_obs, theta_true) + np.random.randn(n) * epsilon

# 定义自定义似然函数
def log_likelihood(y_obs, y_pred, epsilon):
    return pm.Normal.logp(y_obs, mu=y_pred, sigma=epsilon)

# 定义模型
with pm.Model() as model:
    # 定义参数的先验分布
    theta = [pm.Normal(f'theta_{i}', mu=0.0, sigma=1.0) for i in range(5)]
    # 将参数限制在一定范围内
    theta[0] = pm.Uniform('theta_0', lower=0.0, upper=2.0)
    theta[1] = pm.Uniform('theta_1', lower=0.0, upper=4.0)
    theta[2] = pm.Uniform('theta_2', lower=-2.0, upper=0.0)
    theta[3] = pm.Uniform('theta_3', lower=-1.0, upper=1.0)
    theta[4] = pm.Uniform('theta_4', lower=1.0, upper=5.0)
    
    # 定义似然函数
    y_pred = model(x_obs, theta)
    likelihood = pm.DensityDist('likelihood', logp=lambda y_obs, y_pred: log_likelihood(y_obs, y_pred, epsilon), observed={'y_obs': y_obs, 'y_pred': y_pred})
    
    # 进行变分推断
    approx = pm.fit(method='advi')
    trace = approx.sample(1000)

# 输出参数的后验分布的统计信息
pm.summary(trace)
```








```
import pymc3 as pm
import numpy as np
import theano.tensor as tt
import torch

# 自定义似然函数
def log_likelihood(y_obs, x_obs, *theta):
    # 加载训练好的神经网络模型
    model = torch.load('my_model.pt')
    model.eval()

    # 计算模型预测
    with torch.no_grad():
        x = torch.tensor(x_obs).unsqueeze(1).float()
        theta = torch.tensor(theta).unsqueeze(0).float()
        y_pred = model(x, theta).squeeze().numpy()

    # 计算似然函数的对数值
    epsilon = 0.5
    log_likelihood = tt.sum(pm.Normal.dist(mu=y_pred, sigma=epsilon).logp(y_obs))

    return log_likelihood

# 生成观测数据
np.random.seed(42)
n = 50
x_obs = np.linspace(-5, 5, n)
theta_true = np.array([0.5, -1.0, 2.0, 0.0, 1.0])
epsilon = 0.5
y_obs = np.zeros(n)
for i in range(n):
    y_obs[i] = log_likelihood(np.array([y_obs[i]]), np.array([x_obs[i]]), *theta_true)

# 定义模型
with pm.Model() as model:
    # 定义参数的先验分布
    theta = [pm.Uniform(f'theta_{i}', lower=0.0, upper=1.0) for i in range(5)]
    # 定义似然函数
    likelihood = pm.DensityDist('likelihood', log_likelihood, observed={'y_obs': y_obs, 'x_obs': x_obs, 'theta': theta})
    # 进行变分推断
    approx = pm.fit(method='advi')
    trace = approx.sample(1000)

# 输出参数的后验分布的统计信息
pm.summary(trace)
```

在上述代码中，我们首先自定义了一个log_likelihood函数，其中使用了torch库加载训练好的神经网络模型，并将其应用于参数和自变量上来计算模型预测。然后，我们使用pm.Uniform来定义5个参数的先验分布，并将其作为参数传递给pm.DensityDist来定义自定义的似然函数。在定义pm.DensityDist时，我们使用observed参数来将观测数据和自变量传递给似然函数。最后，我们使用pm.fit函数进行变分推断，并使用pm.summary函数输出参数的后验分布的统计信息。需要注意的是，在使用PyTorch模型计算模型预测时，需要

